# Assignment 4.1: Tilt Alarm System

## Part A4.1

### Buzzer sensor module used: KY006 (Passive) or KY012- (Active)

#### Include placeholder

In [29]:
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
import socket

In [ ]:
FAILURE = -1
SUCCESS = 0
SERVER_FREQ = 2500

In [30]:
#creat Pmod_IO object to be used for server. Wired to + of buzzer sensor to PMODB
gpio_pin_1 = Pmod_IO(base.PMODB,1,'out')
#create Pmod_IO object to be used for client. Wired to + of buzzer sensor to PMODA
gpio_pin_2 = Pmod_IO(base.PMODA,1,'out')

In [31]:
#store these pins in a dictionary to easily refrence based on server client
pins = {
    'server'   : gpio_pin_1, 
    'client'   : gpio_pin_2,
}

In [32]:
# definition to turn buzzer off
def turn_buzzer_off(index):
    pins[index].write(0)

In [33]:
# def function to test buzzer
def turn_buzzer_on(index):
    pins[index].write(1)

In [38]:
# def function to turn buzzer on for particular frequency
def turn_alarm_on(freq, index):
    if freq < 0:
        print("frequency should be > 0")
        return FAILURE
    if index not in pins:
        print("index passed does not exist {}".format(index))
        return FAILURE

    turn_buzzer_on(index)
    time.sleep((1/(2*freq)))
    turn_buzzer_off(index)
    time.sleep((1/(2*freq)))
    
    return SUCCESS

In [40]:
freq = 15000 #Hz
index = 'server'
count = 0
while count < 2000: 
    turn_alarm_on(freq, index)
    count +=1

In [43]:
from enum import Enum
class Button_State(Enum):
    Connect = 0
    Alarm = 1
    Disconnect = 2

Button_cur_State = Button_State.Connect

In [ ]:
address = ('localhost', 6000)     # family is deduced to be 'AF_INET'

In [45]:
def server(_lock):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
    # bind the socket to a public host, and a well-known port
    sock.bind(("127.0.0.1", 33334))
    
    #Always be a listener
    sock.listen(1)
    
    while True:
        # 2: Accept connections
        clientsocket, addr = sock.accept()
        print ('Got connection from', addr )
        
        # 3: Receive bytes from the connection 
        msg = clientsocket.recv(1024).decode()
        print ("Message received {}".format(msg.decode()))
        
        if msg == 'sound_alarm':
            #use lock
            _lock.acquire(True)
            Button_cur_State = Button_State.Alarm
            _lock.release()
            
        sound_alarm = Button_cur_State
        while Button_cur_State == Button_State.Alarm:
            turn_alarm_on(SERVER_FREQ, 'server')
            _lock.acquire(True)
            sound_alarm = Button_cur_State
            _lock.release()


In [46]:
def client(_lock):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(('127.0.0.1', 33334))
    
    cond = True
    
    while cond:
        _lock.acquire(True)
        Button_State = Button_cur_State
        _lock.release()
        if Button_State == Button_State.Alarm:
            sock.send('sound_alarm'.encode())
        elif Button_State == Button_State.Disconnect:
            cond = False
            sock.close()

In [50]:
import asyncio
btn_cond = True

async def get_btns(_loop):
    global btn_cond
    while btn_cond:
        await asyncio.sleep(0.01)
        btn = btns.read()
        if btn == 0x01:
            Button_cur_State = Button_State.Connect
            print("Connect button pressed")
        elif btn == 0x02:
            Button_cur_State = Button_State.Alarm
            print("Alarm button pressed")
        elif btn == 0x04:
            Button_cur_State = Button_State.Disconnect
            print("Disconnect button pressed")
            btn_cond = False
            _loop.stop()
        else:
            print("Invalid button pressed")
            
#loop = asyncio.new_event_loop()
#loop.create_task(flash_leds())
#loop.create_task(get_btns(loop))
#loop.run_forever()
#loop.close()        